In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2448900/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_4_49_9,0.999994,0.545079,0.444810,0.999992,0.000003,0.270061,0.388922,0.000003,0.000862,0.001752,1.000003,0.001813,219.388294,355.304440,"Hidden Size=[24], regularizer=0.02, learning_r..."
1,model_4_49_8,0.999994,0.545037,0.444770,0.999992,0.000003,0.270086,0.388950,0.000003,0.000869,0.001760,1.000003,0.001821,219.370567,355.286713,"Hidden Size=[24], regularizer=0.02, learning_r..."
2,model_4_49_7,0.999994,0.544994,0.444730,0.999992,0.000003,0.270111,0.388978,0.000003,0.000876,0.001768,1.000003,0.001830,219.351723,355.267869,"Hidden Size=[24], regularizer=0.02, learning_r..."
3,model_4_49_6,0.999994,0.544952,0.444690,0.999993,0.000003,0.270136,0.389006,0.000003,0.000882,0.001777,1.000003,0.001839,219.332141,355.248287,"Hidden Size=[24], regularizer=0.02, learning_r..."
4,model_4_49_5,0.999994,0.544908,0.444649,0.999993,0.000003,0.270162,0.389035,0.000003,0.000889,0.001786,1.000003,0.001848,219.311606,355.227752,"Hidden Size=[24], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,model_2_0_1,0.075370,0.021301,-0.139052,0.014556,0.472796,0.580998,0.797929,0.740731,0.391255,0.687602,7.703565,0.711736,67.498181,113.737694,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4976,model_8_0_0,0.063018,0.014462,-0.059700,-0.164538,0.479113,0.585058,0.742342,0.536310,0.411033,0.692180,1.295353,0.716474,243.471639,413.016523,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
4977,model_2_0_0,0.061436,0.006635,-0.159738,0.011824,0.479922,0.589704,0.812421,0.742785,0.394254,0.692764,7.804591,0.717079,67.468265,113.707778,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4978,model_1_6_5,0.060607,0.217127,0.858666,-11.064040,0.480346,0.464747,0.099008,2.337113,0.103817,0.693070,-1.270201,0.717395,35.466499,59.286854,"Hidden Size=[4], regularizer=0.02, learning_ra..."
